In [30]:
import pandas as pd
import numpy as np
import time

In [6]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

----

#### Read reference data into dataframe

In [9]:
df = pd.read_csv('../reference_w_loc.csv')

In [13]:
# read the sports dataset with GPS coordinates
sports_df = pd.read_csv('msf_w_coord.csv')

In [14]:
sports_df.head(3)

,urban_area,country,pop_rank,population,major_4_cnt,major_6_cnt,NFL,MLB,NBA,NHL,MLS,CFL,ua_lat,ua_lon
0,New York City,United States,1,"20,685,000",9,11,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3],Red Bulls New York City FC,�,40.730646,-73.986614
1,Los Angeles,United States,2,"15,135,000",8,10,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks,Galaxy Los Angeles FC[note 5],�,34.054935,-118.244476
2,San Francisco Bay Area,United States,7,"5,955,000",6,7,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7],Earthquakes,�,37.788497,-122.355847


---

#### haversine distance

In [16]:
def distance_np(c1, c2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1 = c1 
    lon2, lat2 = c2
    
    lon2_,lat2_ = lon2,lat2
    
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    miles = km/1.609344
    return miles

----

In [35]:
t0 = time.time()

sports_df['distance_from_target'] = float("inf")

df['closest_metro_name'] = "NA"
df['closest_metro_distance'] = float("inf")
df['closest_metro_pop'] = float("inf")
df['closest_metro_m4'] = float("inf")
df['closest_metro_m6'] = float("inf")

for index,row in df.iterrows():
    longitude = row['geocoded_longitude']
    latitude = row['geocoded_latitude']
    
    sports_df_with_distance_to_current_row = sports_df.assign(distance_from_target = distance_np((latitude,longitude),(sports_df['ua_lat'],sports_df['ua_lon'])))
    smallest_distance_row = sports_df_with_distance_to_current_row.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_metro_name = smallest_distance_row['urban_area']
    closest_metro_distance = smallest_distance_row['distance_from_target']
    closest_metro_pop = smallest_distance_row['population']
    closest_metro_m4 = smallest_distance_row['major_4_cnt']
    closest_metro_m6 = smallest_distance_row['major_6_cnt']
    
    df.ix[index, 'closest_metro_name'] = closest_metro_name
    df.ix[index, 'closest_metro_distance'] = closest_metro_distance
    df.ix[index, 'closest_metro_pop'] = closest_metro_pop
    df.ix[index, 'closest_metro_m4'] = closest_metro_m4
    df.ix[index, 'closest_metro_m6'] = closest_metro_m6
    
t1 = time.time()
print t1-t0

415.463745117


In [36]:
df.head()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country,closest_metro_name,closest_metro_distance,closest_metro_pop,closest_metro_m4,closest_metro_m6
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.661256,-91.529911,Iowa City,Iowa,us,St. Louis,88.981643,"2,195,000",2.0,2.0
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.034993,-87.922497,Milwaukee,Wisconsin,us,Milwaukee,0.000000,"1,415,000",2.0,2.0
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.215094,-123.100707,Shelton,Washington,us,Seattle,55.154815,"3,475,000",2.0,3.0
3,3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,unknown,19950510,38.951883,-92.333737,Columbia,Missouri,us,Minneapolis Saint Paul,56.543882,"2,795,000",4.0,5.0
4,4,Anonymous caller repts. sighting 4 ufo&apos;s ...,NaN,"Seattle, WA",19950614,unknown,19950611,47.603832,-122.330062,Seattle,Washington,us,Seattle,0.000000,"3,475,000",2.0,3.0


#### save df to pick / read df from pickle

In [37]:
# df.to_pickle('../pickle_files/df_ref_w_loc_and_sports.pkl')
# df = pd.read_pickle('../pickle_files/df_ref_w_loc_and_sports.pkl')

---

In [38]:
df.to_csv('reference_w_sports.csv', index=False)